In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

from glob import glob
import json
import csv
import os

import cv2
import numpy as np

In [3]:
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import LeakyReLU, ELU, Dropout

Using TensorFlow backend.


In [4]:
from keras.optimizers import Adam
from keras.layers import Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
from utils import read_samples
from utils import ImageGenerator
from utils import CenterImageGenerator

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
ROOT_DIR = os.path.join(os.getcwd(), '../')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [8]:
samples = read_samples(DATA_DIR)
samples.pop(0)

Loaded 8037 samples.


['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

In [9]:
def append_path(line):
    line[0] = os.path.join(DATA_DIR, 'sample-data', line[0].strip())
    line[1] = os.path.join(DATA_DIR, 'sample-data', line[1].strip())
    line[2] = os.path.join(DATA_DIR, 'sample-data', line[2].strip())
    return line

In [10]:
samples = [append_path(line) for line in samples]

In [11]:
samples[0]

['/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/center_2016_12_01_13_30_48_287.jpg',
 '/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/left_2016_12_01_13_30_48_287.jpg',
 '/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/right_2016_12_01_13_30_48_287.jpg',
 ' 0',
 ' 0',
 ' 0',
 ' 22.14829']

In [26]:
train_set, test_set = train_test_split(samples, test_size=0.2)
train_set, valid_set = train_test_split(train_set, test_size=0.2)

In [29]:
train_iterator = ImageGenerator(train_set, batch_size=32, corr=0.1)
valid_iterator = CenterImageGenerator(valid_set, batch_size=32, 
                                      flip=True, gray=False)
test_iterator = CenterImageGenerator(test_set, batch_size=32, 
                                     gray=False)

In [30]:
def comma_model():
    ch, row, col = 1, 160, 320  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
    model.add(ELU())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
    model.add(ELU())
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")
    return model

In [31]:
model = comma_model()

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 80, 16)        1040      
_________________________________________________________________
elu_1 (ELU)                  (None, 40, 80, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 40, 32)        12832     
_________________________________________________________________
elu_2 (ELU)                  (None, 20, 40, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 20, 64)        51264     
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
__________

In [33]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, 
                               patience=2, verbose=1, mode='auto')
filepath = 'comma.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, 
                save_weights_only=False, mode='auto', period=1)

In [34]:
model.fit_generator(generator=train_iterator, epochs=10, 
                    validation_data=valid_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_steps=len(valid_iterator),
                    callbacks=[early_stopping, checkpoint]
                   )

Epoch 1/10
 14/161 [=>............................] - ETA: 178s - loss: 12.5710

KeyboardInterrupt: 

In [86]:
results = model.evaluate_generator(generator=test_iterator, steps=len(test_iterator))

In [87]:
results

0.020940254012552593